In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import os
import torch
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd
from datasets import Dataset

In [3]:
scratch_cache_dir = "/mnt/fast0/rje41/.cache/huggingface"    
model_path = "EleutherAI/pythia-1.4b-deduped"
os.makedirs(os.path.join(scratch_cache_dir, "hub"), exist_ok=True)
os.makedirs(os.path.join(scratch_cache_dir, "datasets"), exist_ok=True)


In [4]:
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             cache_dir=os.path.join(scratch_cache_dir, "hub")
                                            )

In [5]:
model.config.use_cache = False 
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [6]:
lora_config = LoraConfig(
   r=64,
   lora_alpha=128,
   lora_dropout=0.05,
   bias="none",
   task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 12,582,912 || all params: 1,427,230,720 || trainable%: 0.8816


In [7]:
file_path = '../../datasets/add_sub.csv'

df = pd.read_csv(file_path)
dataset = Dataset.from_pandas(df)

def concat_clean_label(example):
    example["clean_label"] = example["clean"] + str(example["label"])
    return example

dataset = dataset.map(concat_clean_label)

def create_labels_no_mask(example):
    tokenized = tokenizer(example["clean_label"], truncation=True, padding="max_length", max_length=256)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

dataset = dataset.map(create_labels_no_mask, batched=False, remove_columns=['clean', 'label', 'corrupted', 'clean_label'])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [8]:
train_valid = dataset.train_test_split(test_size=0.1)
train_valid_split = train_valid['train'].train_test_split(test_size=0.1)

In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    logging_steps=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    save_strategy="steps", 
    save_steps = 1000,
    max_steps = 10000,
    logging_dir='./logs'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_valid_split['train'],
    # eval_dataset=train_valid_split['test'], 
    tokenizer=tokenizer
    )

trainer.train()

/tmp/ipykernel_2642411/981539509.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: rjeskins (rjeskins-university-of-bath) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,18.462000
2,17.516900
3,16.605100
4,15.632200
5,14.574900
6,13.559000
7,12.618800
8,11.386100
9,10.434500
10,9.248800


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

KeyboardInterrupt: 